# Markov processes

This tutorial explains markov processes and how they can be used to generate musical material.

Running this notebook requires a jupyter kernel that contains the musx package.  See [INSTALL.md](https://github.com/musx-admin/musx/blob/main/INSTALL.md) for directions on how to install musx in your environment.
<hr style="height:1px;color:gray">

Python imports:

In [29]:
import matplotlib.pyplot as plt
from musx import version, setmidiplayer, playfile, Score, Seq, Note, MidiFile, \
    Markov, Choose, fit, interp, pick, between, intempo, keynum, rhythm, odds, playfile
from musx.midi.gm import ChoirAahs, AcousticGrandPiano, Marimba
print(version)

N.N.N


Generate midi files and automatically play them using [fluidsynth](https://www.fluidsynth.org/download/) and the [MuseScore_General.sf3](https://ftp.osuosl.org/pub/musescore/soundfont/MuseScore_General) sound font. See [INSTALL.md](https://github.com/musx-admin/musx/blob/main/INSTALL.md) for how to install a terminal based midi player to use with musx.  If you dont have a player installed you can access the output files in the same directory as this notebook:

In [6]:
setmidiplayer("fluidsynth -iq -g1 /usr/local/sf/MuseScore_General.sf3")

### Markov processes

A *markov process* (markov chain) is a random process whose outcomes are affected by previous choices (<I>conditional probability</I>).

<IMG SRC="img/markov1.jpg">

We can think of this process as a table where each row defines the outcome probabilities of one (or more) past choices. Outcomes from the process are then fed back into the process to determine the next outputs.
    
<IMG SRC="img/markov2.jpg">

* Line one: The probablity of A given A is .5 and the probabilty of B given A is .5.
* Line two: The probability of A given B is .2 and the probability of B given B is .8.

What is the probability of sequence ABA given A? 

<p>P<sub>ba</sub> * P<sub>ab</sub> = .5*.2 = .1</p>

<p>
A Markov process can be easily represented as a <I>transition table</I> that maps inputs to outputs according to probabilities.  The total proability of each transition (row) must equal 1.0, meaning that some outcome in a row must result from the past choice(s).
</p>

<IMG SRC="img/markov3.jpg">

<p>A Markov process is generic -- its outputs can be applied to many types of data sets. In music the data might be pitches, rhythms, harmonic choices, etc.  Because past choices influence subsequent choices the process can <I>mimic</I> many types of musical pattern behavior.</p>

<IMG SRC="img/markov4.jpg">
    

### The Markov pattern

```musx.Markov(rules, stop=None, initial=[])```

The Markov pattern class produces its items according to a dictionary of (nth order) transition rules whose keys and values link past outcomes to a set of possible future outcomes:

<code>{(*past1*, ...): [[<em>next</em>, <em>prob</em>], [<em>next</em>, <em>prob</em>], ...], 
  (*past2*, ...): [[<em>next</em>,<em>prob</em>], [<em>next</em>, <em>prob</em>], ...],
  ...}
</code> 

<!--In first order markov, each key in the dictionary is a 1-tuple holding one past outcomes and the key's value is a list of probability outcome pairs.-->
Here is a first order markov process that outputs a pattern of 'a', 'b', and 'c'. Note that the second rule says that if the previous output is 'b' then 'a' always follows -- you can veryify this by looking at output chain of values:

In [ ]:
rules = {('a'): [['b', 1], ['c', 3]], 
         ('b'): [['a', 1]], 
         ('c'): [['a', 5], ['c', 1], ['b', 2.5]]}
pat = Markov(rules)
data = pat.next(20)
print(data)

For convenience sake, in the case of first order Markov where rules only use one past event (see example above) the rule's key can be the item itself, rather than a tuple containing that item.  In addition, any future value that would otherwise be a list with a weight of 1 can be replaced by just the future value.  

This next example uses both these convenieces to make a simplified version of the rules.

In [ ]:
rules = {'a': ['b', ['c', 3]], 
         'b': ['a'], 
         'c': [['a', 5], 'c', ['b', 2.5]]}
pat = Markov(rules)
data = pat.next(20)
print(data)

### Markov orders and `Markov.analyze()`

One of the most interesting uses of Markov is to model real-world phenomena by analyzing its outcomes (data) to extract the probability of its next outcome given N previous outcomes. Given such an analysis one can then generate chains with characteristics approaching original data. How similar the results will depend on the number of previous outcomes N used to determine the next outcome.

The number N is called the *markov order* of the process, and as the order increases, a markov process will come closer and closer to exactly mimicing the data it is derived from.

The next examples demonstrate the effect of increasing the order of a markov process to create melodies that come closer and closer to the original data.

Markov setup:

In [ ]:
# The melodic pitches to use
melody = [60, 60, 62, 60, 65, 64, 60, 60, 62, 60, 67, 65,
          60, 60, 72, 69, 65, 64, 62, 70, 70, 69, 65, 67, 65]
        
def playmarkov(mpat):
    '''Generates output from a markov pattern to a midi file.'''
    def markovmelody(score, reps, mpat, rate):
        '''Generates notes to a score from a markov pattern.'''
        for _ in range(reps):
            p = mpat.next()
            score.add(Note(time=score.now, pitch=p, duration=rate * 1.5))
            yield rate
    print(f"rules: {mpat.items}")
    seq=Seq()
    score=Score(out=seq)
    score.compose( markovmelody(score, 50, mpat, .2) )
    f=MidiFile("markovmelody.mid", seq).write()
    print(f"Wrote '{f.pathname}'")
    playfile(f.pathname)
    
print(f'playmarkov: {playmarkov}')

### Markov.analyze(data, order)

The following examples call the  `Markov.analyze()` factory method to return patterns with increasing orders of Markov data that will increaseing identify the melody.  At what order do you recognize the melody?

1st order Markov considers one past value, the original melody is only vaguely present:

In [ ]:
markov = Markov.analyze(melody, order=1)
playmarkov(markov)

2nd order Markov considers two past values, some motives are recognizable:

In [ ]:
rules = Markov.analyze(melody, order=2)
playmarkov(rules)

3rd order Markov...getting closer!

In [ ]:
rules = Markov.analyze(melody, order=3)
playmarkov(rules)

4th order Markov, you should recogize the melody by now:

In [ ]:
rules = Markov.analyze(melody, order=4)
playmarkov(rules)

In 5th order Markov the pattern is completely determined since there is only one possible output for each row:

In [ ]:
rules = Markov.analyze(melody, order=5)
playmarkov(rules)

## Markov Examples

### Markov Chanting

In the crudest of terms, a Gregorian Chant is characterized by mostly stepwise motion within a range of modal degrees. From any given tone there is more likelihood that a chant will move a step up or down than leap to a tone further away. The larger the skip, the more unlikely it is to occur. In addition, certain tones, such as the
final and tenor, have more influence over the melody than other tones. For example, in the Dorian mode the tenor note A is occasionally decorated by the B-flat directly above it. This B-flat almost always returns back to the tenor tone. In an authentic mode, the final of the mode also acts as a kind of reflecting boundary
that redirects the melody in the opposite direction. 

We can loosely mimic these stylistic tendencies using a first order Markov process. (The table values were created ad hoc, a much better way would be to analyze a collection of dorian chants...):

In [10]:
chantrules = { 
    ('d4',): [['d4', .1],  ['e4', .35], ['f4', .25], ['g4', .1], ['a4', .15]],
    ('e4',): [['d4', .35], ['e4', .1], ['f4', .35], ['g4', .1], ['a4', 1]],
    ('f4',): [['d4', .2], ['e4', .2], ['f4', .1], ['g4', .2], ['a4', .12]],
    ('g4',): [['d4', .2], ['e4', .1], ['f4', .3], ['g4', .1], ['a4', .3], ['bf4', .2]],
    ('a4',): [['d4', .1], ['e4', .1], ['f4', .2], ['g4', .3], ['a4', .2], ['bf4', .3]],
    ('bf4',): [['a4', 1]]
}

def monk(score, reps, chant, rhy):
    pat = Markov(chant)
    for _ in range(reps):
        k = keynum(pat.next())
        n = Note(time=score.now, pitch=k, duration=rhy)
        score.add(n)
        yield rhy
        
def playchant(rules):
    m=MidiFile.metatrack(ins={0: ChoirAahs})
    t=Seq()
    s=Score(out=t)
    s.compose( monk(s, 50, chantrules, .35) )
    f=MidiFile("chant.mid", [m, t]).write()
    playfile(f.pathname)
    return rules

print(f"OK!")

OK!


In [13]:
playchant(chantrules)

{('d4',): [['d4', 0.1], ['e4', 0.35], ['f4', 0.25], ['g4', 0.1], ['a4', 0.15]],
 ('e4',): [['d4', 0.35], ['e4', 0.1], ['f4', 0.35], ['g4', 0.1], ['a4', 1]],
 ('f4',): [['d4', 0.2], ['e4', 0.2], ['f4', 0.1], ['g4', 0.2], ['a4', 0.12]],
 ('g4',): [['d4', 0.2],
  ['e4', 0.1],
  ['f4', 0.3],
  ['g4', 0.1],
  ['a4', 0.3],
  ['bf4', 0.2]],
 ('a4',): [['d4', 0.1],
  ['e4', 0.1],
  ['f4', 0.2],
  ['g4', 0.3],
  ['a4', 0.2],
  ['bf4', 0.3]],
 ('bf4',): [['a4', 1]]}

In the next example we can add some rhythmic interest by possibly adjusting a note's duration if it is in the tonic triad and also forcing the last note to be the tonic.

In [18]:
def chant_dur(knum, dur):
    pc = knum % 12
    if pc == 2:
        return odds(.7, dur * 2, dur)
    elif pc == 9:
        return odds(.5, dur * 2, dur)
    elif pc == 5:
        return odds(.25, dur * 2, dur)
    return dur

def choir_monk(score, endtime, chant, dur, octave):
    pat = Markov(chant)
    while True:
        k = keynum(pat.next())
        if score.elapsed > endtime and ((k % 12) == 2):
            yield -1
        d = chant_dur(k, dur)
        score.add(Note(time=score.now, pitch=k+octave, amplitude=.8, duration=dur))
        yield dur
print(f'choir_monk: {choir_monk}')

choir_monk: <function choir_monk at 0x12951a290>


For fun will run three monks simultaneously at the interval of a tritone.

In [19]:
t=Seq()
m=MidiFile.metatrack(ins={0: ChoirAahs})
s=Score(out=t)
s.compose([choir_monk(s, 20, chantrules, .8, 6), 
           choir_monk(s, 20, chantrules, .8, 0) ,  
           choir_monk(s, 20, chantrules, .8, -6)])
f=MidiFile("chorus.mid", [m,t]).write()
print(f"Wrote '{f.pathname}'")
playfile(f.pathname)

### Markov Harmony

A Markov process can be used to determine an interesting mix intervals that can be stacked to create different chords.
This example uses an ad hoc table to compose harmony and melody vaguely reminiscent of Messiaen's modes.

In [25]:
harmonyrules = {1: [[3, .4], [4, .4], [6, .1]],
                2: [[2, .2], [3, .4], [4, .4], [6, .1]],
                3: [[1, .2], [2, .6], [4, .4]],
                4: [[2, .2], [3, .4], [4, .4]],
                6: [[2, .4], [3, .2], [4, .2]]
}

def markov_harmonizer (score, length, rules, top, size, upward, rhy, dur):
    pat = Markov(rules)
    for _ in range(length):
        i = pat.next()
        top = fit(top + odds(upward, i, -i), 50, 90)
        k = top
        for _ in range(size):
            m = Note(time=score.now, pitch=k, duration=dur)
            score.add(m)
            k -= pat.next()
        yield rhy

t=Seq()
m=MidiFile.metatrack(ins={0: AcousticGrandPiano}) 
s=Score(out=t)
s.compose(markov_harmonizer(s, 25, harmonyrules, 67, 6, .7, 1.2, 1.2))
f=MidiFile("harmony.mid", [m,t]).write()
print(f"Wrote '{f.pathname}'")
playfile(f.pathname)

### Markov Rhythmic Patterns

Imagine generating a melody for a soloist in which the rhythms are determined by weighted random selection. Even if only a few simple rhythms are used, the 'metric pulse' of the process will be weak since sixteenths and dotted eighths can appear anywhere, only occasionally lining up with the start of a metric pulse. If a tempo curve is also applied the beat becomes even more obscured.

In [43]:
def ranrhythm(s, leng, rhys, lo, hi):
    tcurve = [0, 1, .7, .75, 1, 1]
    for i in range(leng):
        k = between(lo, hi)
        r = rhys.next()
        d = intempo(r, 120) * interp(i / leng, tcurve)
        m = Note(time=s.now, pitch=k, duration=d)
        s.add(m)
        yield d

# rhythms are random
rhythms = Choose([1, .75, .5, .25])
        
t=Seq()
m=MidiFile.metatrack(ins={0: Marimba})
s=Score(out=t)
s.compose(ranrhythm(s, 75, rhythms, 40, 80))
f=MidiFile("rhythm.mid", [m,t]).write()
print(f"Wrote '{f.pathname}'")
playfile(f.pathname)

Wrote 'rhythm.mid'


In the next version a first order markov process produces random variations that support the perception of an underlying pulse, even in the case of a tempo curve warping the timing. Table labels are Q=quarter, E.=dotted eighth, E=eighth, S=sixteenth, and table values are probability weights.

<table>
    <tr> <td> </td>   <td>Q</td>   <td>E.</td>   <td>E</td>  <td>S</td>  </tr>
    <tr> <td>Q</td>   <td>.5</td>  <td>.75</td>  <td>2</td>  <td>2</td>  </tr>
    <tr> <td>E.</td>  <td>0</td>   <td>0</td>    <td>0</td>  <td>1</td>  </tr>
    <tr> <td>E</td>   <td>2</td>   <td>1</td>    <td>2</td>  <td>0</td>  </tr> 
    <tr> <td>S</td>   <td>1</td>   <td>0</td>    <td>2</td>  <td>1</td>  </tr> 
    <tfoot></tfoot>
</table>

In [44]:
rules = {
    1: [[1, .5], [.75, .75], [.5, 2]],
    .75: [.25],
    .5: [[1, 2], .75, [.5, 2]],
    .25: [1, [.5, 2], .25]
}

# ryhthms are 
rhythms = Markov(rules)

t=Seq()
m=MidiFile.metatrack(ins={0: Marimba})
s=Score(out=t)
s.compose(ranrhythm(s, 75, rhythms, 40, 80))
f=MidiFile("rhythm.mid", [m,t]).write()
print(f"Wrote '{f.pathname}'")
playfile(f.pathname)

Wrote 'rhythm.mid'


### Markov Texture

This example is a take on keyboard motions between black keys and white keys that can be found in some of Ligeti's works. THe choice of octives is also made using a markov process.

In [38]:
bwmotion = {
    0:  [[0, .5],  [2, 2],   [4, 1.5], [5, 1],   [7, .5],   [9, .5],  [11, .5],  [1, .2], [3, .1], [6, .1], [8, .1],  [10, .1]],
    1:  [[1, .5],  [3, 2],   [6, 1.5], [8, 1],   [10, 1],   [0, .2],  [2, .2],   [4, .1], [5, .1], [7, .1], [9, .1],  [11, .1]],
    2:  [[0, 2],   [2, .5],  [4, 2],   [5, 1.5], [7, 1],    [9, .5],  [1, .2],   [3, .2], [6, .1], [8, .1], [10, .1], [11, .1]],
    3:  [[1, 2],   [3, .5],  [6, 1.5], [8, 1],   [10, .5],  [0, .1],  [2, .2],   [4, .2], [5, .1], [7, .1], [9, .1],  [11, .1]],
    4:  [[0, 1.5], [2, 2],   [4, .5],  [5, 2],   [7, 1.5],  [9, 1],   [11, .5],  [1, .1], [3, .2], [6, .2], [8, .1],  [10, .1]],
    5:  [[0, 1],   [2, 1.5], [4, 2],   [5, .5],  [7, 2],    [9, 1.5], [11, 1],   [1, .1], [3, .2], [6, .2], [8, .1],  [10, .1]],
    6:  [[1, 1.5], [3, 2],   [6, .5],  [8, 2],   [10, 1.5], [0, .1],  [2, .1],   [4, .1], [5, .2], [7, .2], [9, .1],  [11, .1]],
    7:  [[0, .5],  [2, 1],   [4, 1.5], [5, 2],   [7, .5],   [9, 2],   [11, 1.5], [1, .1], [3, .1], [6, .2], [8, .2],  [10, .1]],
    8:  [[1, 1],   [3, 1.5], [6, 2],   [8, .5],  [10, 2],   [0, .1],  [2, .1],   [4, .1], [5, .1], [7, .2], [9, .2],  [11, .1]],
    9:  [[0, .5],  [2, .4],  [4, 1],   [5, 1.5], [7, 2],    [9, .5],  [11, 2],   [1, .1], [3, .1], [6, .1], [8, .2],  [10, .2]],
    10: [[1, .5],  [3, 1],   [6, 1.5], [8, 2],   [10, .5],  [0, .1],  [2, .1],   [4, .1], [5, .1], [7, .1], [9, .2],  [11, .2]],
    11: [[0, .5],  [2, .5],  [4, .5],  [5, 1],   [7, 1.5],  [9, 2],   [11, .5],  [1, .1], [3, .1], [6, .1], [8, .1],  [10, .2]]
}

bwoctaves = {
    'c3': [['c3', 2],   ['c4', 1],  ['c5', .5], ['c6', .25]],
    'c4': [['c3', 1],   ['c4', 2],  ['c5', 1],  ['c6', .5]],
    'c5': [['c3', .5],  ['c4', 1],  ['c5', 2],  ['c6', 1]],
    'c6': [['c3', .25], ['c4', .5], ['c5', 1],  ['c6', 2]]
}

def bwkeys(score, length, octlist, intlist, rate, chan):
    ints = Markov(intlist)
    octs = Markov(octlist)
    reps = 0
    octv = 0
    for _ in range(length):
        if reps == 0:
            reps = pick(4, 8, 12, 16)
            octv = keynum(octs.next())
        intr = ints.next()
        n = Note(time=score.now, duration=rate * 1, pitch=octv + intr, instrument=chan)
        score.add(n)
        reps = reps - 1
        yield rate

print(f'bwkeys: {bwkeys}')

bwkeys: <function bwkeys at 0x1296c7520>


Listen to one voice:

In [41]:
t=Seq()
m=MidiFile.metatrack(ins={0: Marimba})
s=Score(out=t)
s.compose(bwkeys(s, 120, bwoctaves, bwmotion, .125, 0))
f=MidiFile("texture.mid", [m,t]).write()
print(f"Wrote '{f.pathname}'")
playfile(f.pathname)

Wrote 'texture.mid'


Listen to two voices:

In [40]:
t=Seq()
m=MidiFile.metatrack(ins={0: Marimba, 1: Marimba})
s=Score(out=t)
s.compose([[0, bwkeys(s, 120, bwoctaves, bwmotion, .125, 0)],
          [2, bwkeys(s, 120, bwoctaves, bwmotion, .125, 1)]])
f=MidiFile("texture.mid", [m,t]).write()
print(f"Wrote '{f.pathname}'")
playfile(f.pathname)